In [9]:
import pandas as pd
import numpy as np

In [10]:
# data
#INPUT_NAME='train_wikipedia'
INPUT_NAME='train_civil'

df=pd.read_csv("../data/"+INPUT_NAME+ ".csv")


In [11]:
# filter toxic comments
df_toxic = df.loc[df.toxic==1, 'comment_text']

In [12]:
# save as df
df_toxic.to_csv('../data/toxic_' + INPUT_NAME + '.csv')

In [ ]:
# list of badwords
toxic_words = []